## ETL with API data from the New York Times
This project will include:
1. **Retrieval** of data from an API
2. **Conversion** of data structure from JSON to other data formats
3. **Modification** of the data
4. **Writing** of the new file to a MySQL database
5. **Generation** of a brief summary of the data file

These processes represent sub-components of ETL processes, which are necessary tools for acquiring clean data.

### 1. Retrieval

#### Loading Libraries

In [9]:
import json
import requests
import requests.exceptions
import pandas as pd

#### URL Request Function

In [10]:
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

#### Practice Retrieval - The Godfather

In [11]:
base_url = "https://api.nytimes.com/svc/movies/v2/"

add_url = "reviews/search.json?query="

search = "godfather"

key = "&api-key=yBRfoq3WLAGIPYJU0liEb0sixqpaHKd2"

god_url = base_url + add_url + search + key

god_rev = get_api_response(god_url, "dataframe")

In [12]:
god_rev

,status,copyright,has_more,num_results,results
0,OK,Copyright (c) 2022 The New York Times Company....,False,10,"[{'display_title': 'The Black Godfather', 'mpa..."


#### Retrieval - Tarantino Films

In [13]:
search = "tarantino"

tar_url = base_url + add_url + search + key

tar_revs = get_api_response(tar_url, "dataframe")

In [14]:
tar_revs

,status,copyright,has_more,num_results,results
0,OK,Copyright (c) 2022 The New York Times Company....,False,11,[{'display_title': 'Once Upon a Time ... in Ho...


This dataframe only contains a few variables and doesn't tell us much about the movie reviews.

I located more information about the data returned from the NYT Movie Reviews API from this link: https://developer.nytimes.com/docs/movie-reviews-api/1/routes/reviews/search.json/get

I realized the data would require some transformation.

### 2. Conversion

#### Converting Results Column

Upon further review I noticed that the results column contained an array. It violates the first normal form to have multiple data points at one observation point in a data base so I decided to expand this data. 

In [15]:
tar_revs["results"][0]

[{'display_title': 'Once Upon a Time ... in Hollywood',
  'mpaa_rating': 'R',
  'critics_pick': 1,
  'byline': 'A.O. SCOTT',
  'headline': '‘Once Upon a Time … in Hollywood’ Review: We Lost It at the Movies',
  'summary_short': 'Leonardo DiCaprio and Brad Pitt star as midlevel entertainment industry workers whose relationship forms the core of Quentin Tarantino’s look at the movie past.',
  'publication_date': '2019-07-24',
  'opening_date': '2019-07-26',
  'date_updated': '2019-08-24 16:44:02',
  'link': {'type': 'article',
   'url': 'https://www.nytimes.com/2019/07/24/movies/once-upon-a-time-in-hollywood-review.html',
   'suggested_link_text': 'Read the New York Times Review of Once Upon a Time ... in Hollywood'},
  'multimedia': {'type': 'mediumThreeByTwo210',
   'src': 'https://static01.nyt.com/images/2019/07/24/arts/24onceupon1/24onceupon1-mediumThreeByTwo210.jpg',
   'height': 140,
   'width': 210}},
 {'display_title': 'The Hateful Eight',
  'mpaa_rating': 'R',
  'critics_pick': 

This data is a bundle of dictionaries, essentially a JSON string, so I used the "json_normalize" function through the pandas package to convert the data into a data frame.

In [36]:
tar_revs2 = pd.json_normalize(tar_revs["results"][0])
tar_revs2.head(10)

,display_title,mpaa_rating,critics_pick,byline,headline,summary_short,publication_date,opening_date,date_updated,link.type,link.url,link.suggested_link_text,multimedia.type,multimedia.src,multimedia.height,multimedia.width,multimedia
0,Once Upon a Time ... in Hollywood,R,1,A.O. SCOTT,‘Once Upon a Time … in Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article,https://www.nytimes.com/2019/07/24/movies/once...,Read the New York Times Review of Once Upon a ...,mediumThreeByTwo210,https://static01.nyt.com/images/2019/07/24/art...,140.0,210.0,NaN
1,The Hateful Eight,R,0,A. O. SCOTT,Review: Quentin Tarantino’s ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article,https://www.nytimes.com/2015/12/25/movies/revi...,Read the New York Times Review of The Hateful ...,mediumThreeByTwo210,https://static01.nyt.com/images/2015/12/25/art...,140.0,210.0,NaN
2,Django Unchained,R,1,A. O. SCOTT,"The Black, the White and the Angry","“Django Unchained” is pulpy, digressive, jokey...",2012-12-24,2012-12-25,2017-11-02 04:18:16,article,https://www.nytimes.com/2012/12/25/movies/quen...,Read the New York Times Review of Django Uncha...,NaN,NaN,NaN,NaN,NaN
3,Inglourious Basterds,R,0,MANOHLA DARGIS,Tarantino Avengers in Nazi Movieland,“Inglourious Basterds” is simply another testa...,2009-08-20,2009-08-21,2017-11-02 04:18:10,article,https://www.nytimes.com/2009/08/21/movies/21in...,Read the New York Times Review of Inglourious ...,NaN,NaN,NaN,NaN,NaN
4,Grindhouse,R,0,A. O. SCOTT,Back to the (Double) Feature,“Grindhouse” is soaked in bloody nostalgia for...,2007-04-06,2007-04-06,2017-11-02 04:18:05,article,https://www.nytimes.com/2007/04/06/movies/06gr...,Read the New York Times Review of Grindhouse,NaN,NaN,NaN,NaN,NaN
5,Kill Bill Vol. 2,R,1,Elvis Mitchell,"Vengeance Still Mine, Saith the Lethal Bride",When a writer-director — Cameron Crowe or Paul...,2004-04-16,2004-04-16,2017-11-02 04:18:03,article,https://www.nytimes.com/2004/04/16/movies/film...,Read the New York Times Review of Kill Bill Vo...,NaN,NaN,NaN,NaN,NaN
6,Kill Bill Vol. 1,R,0,A. O. Scott,Blood Bath & Beyond,"Quentin Tarantino's fourth movie, his first in...",2003-10-10,2003-10-10,2017-11-02 04:18:02,article,https://www.nytimes.com/2003/10/10/movies/film...,Read the New York Times Review of Kill Bill Vo...,NaN,NaN,NaN,NaN,NaN
7,Jackie Brown,TV-MA,0,Janet Maslin,Jackie Brown (Movie),"Bail bondsman, smuggler and arms dealer, via T...",1997-12-24,1997-12-25,2017-11-02 04:17:54,article,https://www.nytimes.com/1997/12/24/movies/film...,Read the New York Times Review of Jackie Brown,NaN,NaN,NaN,NaN,NaN
8,Four Rooms,R,0,Janet Maslin,FOUR ROOMS (MOVIE),"Short films, set in crumbling hotel. Self-indu...",1995-12-26,1995-12-25,2017-11-02 04:17:49,article,https://www.nytimes.com/1995/12/26/movies/film...,Read the New York Times Review of Four Rooms,NaN,NaN,NaN,NaN,NaN
9,Pulp Fiction,R,1,Janet Maslin,PULP FICTION (MOVIE),Director Quentin Tarantino's linked vignettes ...,1994-09-23,1994-10-14,2017-11-02 04:17:47,article,https://www.nytimes.com/1994/09/23/movies/film...,Read the New York Times Review of Pulp Fiction,NaN,NaN,NaN,NaN,NaN


### 3. Modification

The data we have retrieved and converted into a dataframe should be adapted to present **cleaner**. We can clean up the number of columns as well as its values and column names by using the **pandas package**.

#### Number of Columns

In [37]:
tar_revs2.drop(tar_revs2.columns[10:17], axis=1, inplace=True)
tar_revs2

,display_title,mpaa_rating,critics_pick,byline,headline,summary_short,publication_date,opening_date,date_updated,link.type
0,Once Upon a Time ... in Hollywood,R,1,A.O. SCOTT,‘Once Upon a Time … in Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,The Hateful Eight,R,0,A. O. SCOTT,Review: Quentin Tarantino’s ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article
2,Django Unchained,R,1,A. O. SCOTT,"The Black, the White and the Angry","“Django Unchained” is pulpy, digressive, jokey...",2012-12-24,2012-12-25,2017-11-02 04:18:16,article
3,Inglourious Basterds,R,0,MANOHLA DARGIS,Tarantino Avengers in Nazi Movieland,“Inglourious Basterds” is simply another testa...,2009-08-20,2009-08-21,2017-11-02 04:18:10,article
4,Grindhouse,R,0,A. O. SCOTT,Back to the (Double) Feature,“Grindhouse” is soaked in bloody nostalgia for...,2007-04-06,2007-04-06,2017-11-02 04:18:05,article
5,Kill Bill Vol. 2,R,1,Elvis Mitchell,"Vengeance Still Mine, Saith the Lethal Bride",When a writer-director — Cameron Crowe or Paul...,2004-04-16,2004-04-16,2017-11-02 04:18:03,article
6,Kill Bill Vol. 1,R,0,A. O. Scott,Blood Bath & Beyond,"Quentin Tarantino's fourth movie, his first in...",2003-10-10,2003-10-10,2017-11-02 04:18:02,article
7,Jackie Brown,TV-MA,0,Janet Maslin,Jackie Brown (Movie),"Bail bondsman, smuggler and arms dealer, via T...",1997-12-24,1997-12-25,2017-11-02 04:17:54,article
8,Four Rooms,R,0,Janet Maslin,FOUR ROOMS (MOVIE),"Short films, set in crumbling hotel. Self-indu...",1995-12-26,1995-12-25,2017-11-02 04:17:49,article
9,Pulp Fiction,R,1,Janet Maslin,PULP FICTION (MOVIE),Director Quentin Tarantino's linked vignettes ...,1994-09-23,1994-10-14,2017-11-02 04:17:47,article


#### Column Names

In [38]:
tar_revs2.columns

column_name_map = {"display_title" : "Movie Name",
                   "mpaa_rating" : "MPAA Rating",
                   "critics_pick" : "Critics Pick",
                   "byline" : "Byline",
                   "headline" : "Headline",
                   "summary_short" : "Summary",
                   "publication_date" : "Publication Date",
                   "opening_date" : "Opening Date",
                   "date_updated" : "Date Updated",
                   "link.type" : "Link Type"
                  }

tar_revs2.rename(columns=column_name_map, inplace=True)
tar_revs2

,Movie Name,MPAA Rating,Critics Pick,Byline,Headline,Summary,Publication Date,Opening Date,Date Updated,Link Type
0,Once Upon a Time ... in Hollywood,R,1,A.O. SCOTT,‘Once Upon a Time … in Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,The Hateful Eight,R,0,A. O. SCOTT,Review: Quentin Tarantino’s ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article
2,Django Unchained,R,1,A. O. SCOTT,"The Black, the White and the Angry","“Django Unchained” is pulpy, digressive, jokey...",2012-12-24,2012-12-25,2017-11-02 04:18:16,article
3,Inglourious Basterds,R,0,MANOHLA DARGIS,Tarantino Avengers in Nazi Movieland,“Inglourious Basterds” is simply another testa...,2009-08-20,2009-08-21,2017-11-02 04:18:10,article
4,Grindhouse,R,0,A. O. SCOTT,Back to the (Double) Feature,“Grindhouse” is soaked in bloody nostalgia for...,2007-04-06,2007-04-06,2017-11-02 04:18:05,article
5,Kill Bill Vol. 2,R,1,Elvis Mitchell,"Vengeance Still Mine, Saith the Lethal Bride",When a writer-director — Cameron Crowe or Paul...,2004-04-16,2004-04-16,2017-11-02 04:18:03,article
6,Kill Bill Vol. 1,R,0,A. O. Scott,Blood Bath & Beyond,"Quentin Tarantino's fourth movie, his first in...",2003-10-10,2003-10-10,2017-11-02 04:18:02,article
7,Jackie Brown,TV-MA,0,Janet Maslin,Jackie Brown (Movie),"Bail bondsman, smuggler and arms dealer, via T...",1997-12-24,1997-12-25,2017-11-02 04:17:54,article
8,Four Rooms,R,0,Janet Maslin,FOUR ROOMS (MOVIE),"Short films, set in crumbling hotel. Self-indu...",1995-12-26,1995-12-25,2017-11-02 04:17:49,article
9,Pulp Fiction,R,1,Janet Maslin,PULP FICTION (MOVIE),Director Quentin Tarantino's linked vignettes ...,1994-09-23,1994-10-14,2017-11-02 04:17:47,article


#### Values

In [39]:
tar_revs2['Byline'] = tar_revs2['Byline'].str.title()
tar_revs2['Headline'] = tar_revs2['Headline'].str.title()

tar_revs2

,Movie Name,MPAA Rating,Critics Pick,Byline,Headline,Summary,Publication Date,Opening Date,Date Updated,Link Type
0,Once Upon a Time ... in Hollywood,R,1,A.O. Scott,‘Once Upon A Time … In Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,The Hateful Eight,R,0,A. O. Scott,Review: Quentin Tarantino’S ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article
2,Django Unchained,R,1,A. O. Scott,"The Black, The White And The Angry","“Django Unchained” is pulpy, digressive, jokey...",2012-12-24,2012-12-25,2017-11-02 04:18:16,article
3,Inglourious Basterds,R,0,Manohla Dargis,Tarantino Avengers In Nazi Movieland,“Inglourious Basterds” is simply another testa...,2009-08-20,2009-08-21,2017-11-02 04:18:10,article
4,Grindhouse,R,0,A. O. Scott,Back To The (Double) Feature,“Grindhouse” is soaked in bloody nostalgia for...,2007-04-06,2007-04-06,2017-11-02 04:18:05,article
5,Kill Bill Vol. 2,R,1,Elvis Mitchell,"Vengeance Still Mine, Saith The Lethal Bride",When a writer-director — Cameron Crowe or Paul...,2004-04-16,2004-04-16,2017-11-02 04:18:03,article
6,Kill Bill Vol. 1,R,0,A. O. Scott,Blood Bath & Beyond,"Quentin Tarantino's fourth movie, his first in...",2003-10-10,2003-10-10,2017-11-02 04:18:02,article
7,Jackie Brown,TV-MA,0,Janet Maslin,Jackie Brown (Movie),"Bail bondsman, smuggler and arms dealer, via T...",1997-12-24,1997-12-25,2017-11-02 04:17:54,article
8,Four Rooms,R,0,Janet Maslin,Four Rooms (Movie),"Short films, set in crumbling hotel. Self-indu...",1995-12-26,1995-12-25,2017-11-02 04:17:49,article
9,Pulp Fiction,R,1,Janet Maslin,Pulp Fiction (Movie),Director Quentin Tarantino's linked vignettes ...,1994-09-23,1994-10-14,2017-11-02 04:17:47,article


In [25]:
len(list(tar_revs2.columns))

10

### 4. Writing to MySQL

We're now going to write the data we have extracted and transformed from the New York Times API to a local MySQL database. We will store the data in MongoDB also to preserve it in its original form.

#### Libraries

In [1]:
import os
import json
import numpy
import datetime
import pandas as pd

import pymongo
from sqlalchemy import create_engine

#### Create Connection Variables for MySQL and MongoDB

In [3]:
host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

user_id = "mlane4"
pwd = "reppiDgiB#7891"

src_dbname = "nyt_movie_review"
dst_dbname = "nyt_movie_reviews"

#### Write Functions

In [6]:
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Retrieve and Write JSON Data

In [51]:
df2 = tar_revs2.to_json(orient = 'records') 

json_string = df2

df2

with open('json_tarantino.json', 'w') as outfile:
    outfile.write(json_string)

#### Populate MongoDB Database

In [52]:
port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"tarantino_reviews" : 'json_tarantino.json'
             }

for file in json_files:
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)
        #print(f"{file} was successfully loaded.")

        
client.close()

#### Extracting MongoDB Data into DFs

In [56]:
query = {}
port = ports["mongo"]
collection = "tarantino_reviews"

df_tarantino = get_mongo_dataframe(None, None, host_name, port, src_dbname, collection, query)
df_tarantino.head(2)

,Movie Name,MPAA Rating,Critics Pick,Byline,Headline,Summary,Publication Date,Opening Date,Date Updated,Link Type
0,Once Upon a Time ... in Hollywood,R,1,A.O. Scott,‘Once Upon A Time … In Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,The Hateful Eight,R,0,A. O. Scott,Review: Quentin Tarantino’S ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article


#### Make a Transformation

In [61]:
df_tarantino.insert(0, "movie_key", range(1, df_tarantino.shape[0]+1))
df_tarantino.head(2)

,movie_key,Movie Name,MPAA Rating,Critics Pick,Byline,Headline,Summary,Publication Date,Opening Date,Date Updated,Link Type
0,1,Once Upon a Time ... in Hollywood,R,1,A.O. Scott,‘Once Upon A Time … In Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,2,The Hateful Eight,R,0,A. O. Scott,Review: Quentin Tarantino’S ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article


#### Load DF into New MySQL Database

In [63]:
dataframe = df_tarantino
table_name = 'tarantino_reviews'
primary_key = 'movie_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### Validation

In [65]:
sql_tarantino = "SELECT * FROM nyt_movie_reviews.tarantino_reviews;"
df_tarantino_reviews = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_tarantino)
df_tarantino_reviews.head(2)

,movie_key,Movie Name,MPAA Rating,Critics Pick,Byline,Headline,Summary,Publication Date,Opening Date,Date Updated,Link Type
0,1,Once Upon a Time ... in Hollywood,R,1,A.O. Scott,‘Once Upon A Time … In Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,2,The Hateful Eight,R,0,A. O. Scott,Review: Quentin Tarantino’S ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article


### 5. Brief Summary

#### Writing Column and Row Stats

In [71]:
print("The # of columns is :", len(df_tarantino_reviews.columns), "and the # of rows is:", len(df_tarantino_reviews))

The # of columns is : 11 and the # of rows is: 11


In [72]:
df_tarantino_reviews

,movie_key,Movie Name,MPAA Rating,Critics Pick,Byline,Headline,Summary,Publication Date,Opening Date,Date Updated,Link Type
0,1,Once Upon a Time ... in Hollywood,R,1,A.O. Scott,‘Once Upon A Time … In Hollywood’ Review: We L...,Leonardo DiCaprio and Brad Pitt star as midlev...,2019-07-24,2019-07-26,2019-08-24 16:44:02,article
1,2,The Hateful Eight,R,0,A. O. Scott,Review: Quentin Tarantino’S ‘The Hateful Eight...,Mr. Tarantino’s western tale of vengeance sett...,2015-12-24,2015-12-30,2017-11-02 04:18:23,article
2,3,Django Unchained,R,1,A. O. Scott,"The Black, The White And The Angry","“Django Unchained” is pulpy, digressive, jokey...",2012-12-24,2012-12-25,2017-11-02 04:18:16,article
3,4,Inglourious Basterds,R,0,Manohla Dargis,Tarantino Avengers In Nazi Movieland,“Inglourious Basterds” is simply another testa...,2009-08-20,2009-08-21,2017-11-02 04:18:10,article
4,5,Grindhouse,R,0,A. O. Scott,Back To The (Double) Feature,“Grindhouse” is soaked in bloody nostalgia for...,2007-04-06,2007-04-06,2017-11-02 04:18:05,article
5,6,Kill Bill Vol. 2,R,1,Elvis Mitchell,"Vengeance Still Mine, Saith The Lethal Bride",When a writer-director — Cameron Crowe or Paul...,2004-04-16,2004-04-16,2017-11-02 04:18:03,article
6,7,Kill Bill Vol. 1,R,0,A. O. Scott,Blood Bath & Beyond,"Quentin Tarantino's fourth movie, his first in...",2003-10-10,2003-10-10,2017-11-02 04:18:02,article
7,8,Jackie Brown,TV-MA,0,Janet Maslin,Jackie Brown (Movie),"Bail bondsman, smuggler and arms dealer, via T...",1997-12-24,1997-12-25,2017-11-02 04:17:54,article
8,9,Four Rooms,R,0,Janet Maslin,Four Rooms (Movie),"Short films, set in crumbling hotel. Self-indu...",1995-12-26,1995-12-25,2017-11-02 04:17:49,article
9,10,Pulp Fiction,R,1,Janet Maslin,Pulp Fiction (Movie),Director Quentin Tarantino's linked vignettes ...,1994-09-23,1994-10-14,2017-11-02 04:17:47,article
